In [16]:
import os
import pandas as pd

In [17]:
data_loc = "/nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data-processed/middle_40"

In [18]:
os.path.isfile(f"{data_loc}/data.csv")

True

In [19]:
df_frames = pd.read_csv(f"{data_loc}/data.csv")

In [20]:
df_frames

,frame,label
0,vNonPorn152#0.jpg,0
1,vNonPorn152#1.jpg,0
2,vNonPorn152#2.jpg,0
3,vNonPorn152#3.jpg,0
4,vNonPorn152#4.jpg,0
...,...,...
31547,vPorn346#35.jpg,1
31548,vPorn346#36.jpg,1
31549,vPorn346#37.jpg,1
31550,vPorn346#38.jpg,1


In [21]:
df_frames["video"] = [frame_name.split("#")[0] for frame_name in df_frames["frame"]]
df_frames

,frame,label,video
0,vNonPorn152#0.jpg,0,vNonPorn152
1,vNonPorn152#1.jpg,0,vNonPorn152
2,vNonPorn152#2.jpg,0,vNonPorn152
3,vNonPorn152#3.jpg,0,vNonPorn152
4,vNonPorn152#4.jpg,0,vNonPorn152
...,...,...,...
31547,vPorn346#35.jpg,1,vPorn346
31548,vPorn346#36.jpg,1,vPorn346
31549,vPorn346#37.jpg,1,vPorn346
31550,vPorn346#38.jpg,1,vPorn346


In [30]:
agg = {"video": "first", "label": "first", "frame": "size"}
df_videos = df_frames[["video", "frame", "label"]]
df_videos = df_videos.groupby("video").aggregate(agg).reset_index(drop=True)
df_videos = df_videos.rename(columns={"frame": "frame_count"})
df_videos

,video,label,frame_count
0,vNonPorn1,0,40
1,vNonPorn10,0,40
2,vNonPorn100,0,40
3,vNonPorn1006,0,40
4,vNonPorn1007,0,40
...,...,...,...
788,vPorn94,1,40
789,vPorn95,1,40
790,vPorn96,1,40
791,vPorn97,1,40


In [31]:
# Videos from which it was not possible to extract 40 frames
df_videos[df_videos["frame_count"] < 40]

,video,label,frame_count
83,vNonPorn1080,0,21
85,vNonPorn1082,0,22
90,vNonPorn1087,0,22
110,vNonPorn1107,0,21
146,vNonPorn1141,0,21
177,vNonPorn1173,0,21
179,vNonPorn1175,0,21
182,vNonPorn1179,0,21
188,vNonPorn1186,0,22


In [34]:
len(df_videos) # 7 videos with 0 frames extracted

793

In [32]:
len(df_frames[df_frames["label"] == 0]) # Number of non-porn frames

15552

In [33]:
len(df_frames[df_frames["label"] == 1]) # Number of porn frames

16000